## RADICAL-Pilot tutorial

This notebook demonstrate a walkthrough example of how to run RADICAL-Pilot on any linux or macOS machine.

The tutorial includes an example of how to a execute workload of tasks using a Bag of Tasks approach (BOT).

`!radical-stack`

```
python               : $HOME/ve/RCT/bin/python3
pythonpath           : 
version              : 3.9
virtualenv           : $HOME/ve/RCT

radical.analytics    : 1.6.7
radical.gtod         : 1.6.7
radical.pilot        : 1.9.2
radical.saga         : 1.8.0
radical.utils        : 1.9.1
```

For more information regarding the examples, please refer our documentation.

In [ ]:
import os
import sys

import radical.pilot as rp
import radical.utils as ru

%load_ext dotenv
%dotenv
print("loaded2")
os.getenv("RADICAL_PILOT_DBURL")
os.getcwd()


## Reprtor for a better visualization
We use a reporter class for a nicer output

In [ ]:
report = ru.Reporter(name='radical.pilot')
report.title('Getting Started (RP version %s)' % rp.version)

## Setting up the session
Create a new session as it is the root object for all other objects in RADICAL-Pilot.

In [ ]:
session = rp.Session()

## Setting up the Pilot and Task manager
Pilots are created via a PilotManager and TaskManager

In [ ]:
pmgr = rp.PilotManager(session=session)
tmgr = rp.TaskManager(session=session)

## Resource configuration

RP supports 2 levels of resource configuration (**pilot** level and **task** level).

### Pilot level resource specification

we first specify how many cores and gpus the pilot requires via pilot description:

In [ ]:
resource = 'local.localhost'
# config = ru.read_json('%s/config.json'
#                             % os.path.dirname(__file__)).get(resource, {})

pd_init = {'resource'      : 'local.localhost',
                   'runtime'       : 30,  # pilot runtime (min)
                   'exit_on_error' : True,
                   'project'       : None,
                   'queue'         : None,
                   'cores'         : 1,
                   'gpus'          : 0
                  }
pdesc = rp.PilotDescription(pd_init)

# pdesc.resource       = 'local.localhost'
# pdesc.runtime        = 30  # pilot runtime (min)
# pdesc.exit_on_error  = True
# pdesc.cores          = 4
# pdesc.gpus           = 1

Now we created the pilot, let's Launch it.

In [ ]:
pilot = pmgr.submit_pilots(pdesc)

It is required to register the pilot in a TaskManager object once the it is launced,

In [ ]:
tmgr.add_pilots(pilot)

### Task level resource specification

For a more fine grained resource specification, RP allows to specify how many cores, threads per cpus and gpus the task requires via task description:

In [ ]:
def shell_task():
    t = rp.TaskDescription()
    t.stage_on_error = True
    t.executable     = '/bin/date'
    t.cpu_processes  = 1
    t.cpu_threads    = 2
    t.gpu_processes  = 1
    t_gpu_threads    = 1
    return t

In [ ]:
bot = list() # Bag of tasks to append the launced tasks to it
for i in range(0, 10):
    task = shell_task()
    bot.append(task)
    report.progress()

report.progress_done()

Submit the previously created task descriptions to the PilotManager. This will trigger the selected scheduler to start
assigning tasks to the pilots.

In [ ]:
tmgr.submit_tasks(bot)

Now Wait for all tasks to reach a final state (**DONE**, **CANCELED** or **FAILED**).

In [ ]:
tmgr.wait_tasks()

Once the wait is finished, clean up the session

In [ ]:
report.header('finalize')
session.close(download=True)
